In [ ]:
import numpy as npy
import pickle
import urllib.request
import os
import tarfile

def loadCifarDataset():
    cifarTar_urlLink = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
    if not os.path.exists("cifar-10-python.tar.gz"):
        urllib.request.urlretrieve(cifarTar_urlLink, "cifar-10-python.tar.gz")
    with tarfile.open("cifar-10-python.tar.gz", "r:gz") as tar:
        tar.extractall()

def cifarBatchLoadandDataExtract(inputfile):
    with open(inputfile, 'rb') as ifile:
        Dictdata = pickle.load(ifile, encoding='bytes')
        Cifarimages = Dictdata[b'data'] / 255.0
        Cifarlabels = npy.array(Dictdata[b'labels'])
        Cifarlabels = npy.eye(10)[Cifarlabels.reshape(-1)]
    return Cifarimages, Cifarlabels

def initialize_parameters(input_size=3072, hidden_size=128, output_size=10):
    npy.random.seed(42)
    weightLayer1 = npy.random.randn(input_size, hidden_size) * npy.sqrt(2.0 / input_size)
    biasValue1 = npy.zeros((1, hidden_size))
    weightLayer2 = npy.random.randn(hidden_size, output_size) * npy.sqrt(2.0 / hidden_size)
    biasValue2 = npy.zeros((1, output_size))
    return weightLayer1, biasValue1, weightLayer2, biasValue2

def reluActivationFunction(Zout):
    return npy.maximum(0, Zout)

def softmaxFunction(Zout):
    exp_Z = npy.exp(Zout - npy.max(Zout, axis=1, keepdims=True))
    return exp_Z / npy.sum(exp_Z, axis=1, keepdims=True)

def forwardPropagationFunction(Xtrainset, weightLayer1, biasValue1, weightLayer2, biasValue2):
    Z1out = npy.dot(Xtrainset, weightLayer1) + biasValue1
    A1actout = reluActivationFunction(Z1out)
    Z2out = npy.dot(A1actout, weightLayer2) + biasValue2
    A2actout = softmaxFunction(Z2out)
    return Z1out, A1actout, Z2out, A2actout

def returnReluDerivative(Zout):
    return (Zout > 0).astype(float)

def computeLossFunction(A2out, Yset, weightLayer1, weightLayer2, lambda_=0.01):
    mtraining = Yset.shape[0]
    crossEntropyVal = -npy.sum(Yset * npy.log(A2out + 1e-8)) / mtraining
    L2regVal = (lambda_ / (2 * mtraining)) * (npy.sum(weightLayer1**2) + npy.sum(weightLayer2**2))
    return crossEntropyVal + L2regVal

def backwardPropagationFunction(Xtrainset, Yset, weightLayer1, biasValue1, weightLayer2, biasValue2, Z1out, A1actout, A2actout, lambda_=0.01):
    mtraining = Xtrainset.shape[0]
    outputGradient_dZ2 = A2actout - Yset
    predict_dW2 = (npy.dot(A1actout.T, outputGradient_dZ2) / mtraining) + (lambda_ * weightLayer2 / mtraining)
    bias_db2 = npy.sum(outputGradient_dZ2, axis=0, keepdims=True) / mtraining
    outputGradient_dZ1 = npy.dot(outputGradient_dZ2, weightLayer2.T) * returnReluDerivative(Z1out)
    predict_dW1 = (npy.dot(Xtrainset.T, outputGradient_dZ1) / mtraining) + (lambda_ * weightLayer1 / mtraining)
    bias_db1 = npy.sum(outputGradient_dZ1, axis=0, keepdims=True) / mtraining
    return predict_dW1, bias_db1, predict_dW2, bias_db2

def update_Dataparameters(weightLayer1, biasValue1, weightLayer2, biasValue2, predict_dW1, bias_db1, predict_dW2, bias_db2, learningRateval=0.005):
    weightLayer1 -= learningRateval * predict_dW1
    biasValue1 -= learningRateval * bias_db1
    weightLayer2 -= learningRateval * predict_dW2
    biasValue2 -= learningRateval * bias_db2
    return weightLayer1, biasValue1, weightLayer2, biasValue2

def trainWeightsandBias(Xtrainset, Ytrainset, noofEpochs=500, learningRateval=0.005, lambda_=0.01):
    weightLayer1, biasValue1, weightLayer2, biasValue2 = initialize_parameters()
    for epoch in range(noofEpochs):
        Z1out, A1actout, Z2out, A2actout = forwardPropagationFunction(Xtrainset, weightLayer1, biasValue1, weightLayer2, biasValue2)
        lossValue = computeLossFunction(A2actout, Ytrainset, weightLayer1, weightLayer2, lambda_)
        predict_dW1, bias_db1, predict_dW2, bias_db2 = backwardPropagationFunction(Xtrainset, Ytrainset, weightLayer1, biasValue1, weightLayer2, biasValue2, Z1out, A1actout, A2actout, lambda_)
        weightLayer1, biasValue1, weightLayer2, biasValue2 = update_Dataparameters(weightLayer1, biasValue1, weightLayer2, biasValue2, predict_dW1, bias_db1, predict_dW2, bias_db2, learningRateval)
        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Loss: {lossValue:.6f}")
    return weightLayer1, biasValue1, weightLayer2, biasValue2

def predictFunction(Xtestset, weightLayer1, biasValue1, weightLayer2, biasValue2):
    _, _, _, A2actout = forwardPropagationFunction(Xtestset, weightLayer1, biasValue1, weightLayer2, biasValue2)
    predictionsValue = npy.argmax(A2actout, axis=1)
    return predictionsValue

if __name__ == "__main__":
    loadCifarDataset()
    Xtrainset, ytrainset = cifarBatchLoadandDataExtract("cifar-10-batches-py/data_batch_1")
    print("Dataset loaded successfully!")
    print("X_train shape:", Xtrainset.shape)
    print("y_train shape:", ytrainset.shape)
    weightLayer1, biasValue1, weightLayer2, biasValue2 = trainWeightsandBias(Xtrainset, ytrainset, noofEpochs=500, learningRateval=0.005, lambda_=0.01)
    predictionsValue = predictFunction(Xtrainset, weightLayer1, biasValue1, weightLayer2, biasValue2)
    labelsActualValue = npy.argmax(ytrainset, axis=1)
    accuracyPercentage = npy.mean(predictionsValue == labelsActualValue) * 100
    print(f"Training Accuracy: {accuracyPercentage:.2f}%")

C:\Users\Suvarna\AppData\Local\Temp\ipykernel_17304\2636927568.py:12: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall()


Dataset loaded successfully!
X_train shape: (10000, 3072)
y_train shape: (10000, 10)
Epoch 0, Loss: 2.519491
Epoch 10, Loss: 2.276183
Epoch 20, Loss: 2.240506
Epoch 30, Loss: 2.212592
Epoch 40, Loss: 2.188195
Epoch 50, Loss: 2.166771
Epoch 60, Loss: 2.148190
Epoch 70, Loss: 2.131369
Epoch 80, Loss: 2.115679
Epoch 90, Loss: 2.100763
